In [1]:
# ==== QCNN sweep with configurable n_qubits + CSV saving ====
import json, time, csv
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import ZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator as Estimator

from qiskit_machine_learning.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit_machine_learning.optimizers import COBYLA

from sklearn.model_selection import train_test_split

# -------------------------
# Helpers
# -------------------------
def _require_power_of_two(n):
    if n < 2 or (n & (n - 1)) != 0:
        raise ValueError(f"n_qubits must be a power of two >= 2, got {n}")

# -------------------------
# Building blocks (conv/pool)
# -------------------------
def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target

def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)

    # even-odd pairs
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : param_index + 3]), [q1, q2])
        qc.barrier()
        param_index += 3

    # odd-next (with wrap) pairs
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : param_index + 3]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()
    qc2 = QuantumCircuit(num_qubits)
    qc2.append(qc_inst, qubits)
    return qc2

def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    return target

def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=(num_qubits // 2) * 3)

    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : param_index + 3]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()
    qc2 = QuantumCircuit(num_qubits)
    qc2.append(qc_inst, range(num_qubits))
    return qc2

# -------------------------
# Data generation for n_qubits
#  - uses a 2 x (n_qubits/2) grid
#  - labels: horizontal pair = -1, vertical pair = +1
# -------------------------
def generate_dataset(num_images, n_qubits):
    _require_power_of_two(n_qubits)
    if n_qubits % 2 != 0:
        raise ValueError("n_qubits must be even (for 2-row grid).")

    width = n_qubits // 2  # 2 rows
    images, labels = [], []

    # Precompute horizontal patterns: two adjacent cells in a row (no wrap)
    # total = 2 * (width - 1)
    hor_patterns = []
    for row in range(2):
        base = row * width
        for c in range(width - 1):
            v = np.zeros(n_qubits)
            v[base + c] = np.pi / 2
            v[base + c + 1] = np.pi / 2
            hor_patterns.append(v)

    # Precompute vertical patterns: same column across two rows
    # total = width
    ver_patterns = []
    for c in range(width):
        v = np.zeros(n_qubits)
        v[c] = np.pi / 2
        v[c + width] = np.pi / 2
        ver_patterns.append(v)

    for _ in range(num_images):
        rng = algorithm_globals.random.integers(0, 2)
        if rng == 0:
            labels.append(-1)
            idx = algorithm_globals.random.integers(0, len(hor_patterns))
            images.append(hor_patterns[idx].copy())
        else:
            labels.append(1)
            idx = algorithm_globals.random.integers(0, len(ver_patterns))
            images.append(ver_patterns[idx].copy())

        # add noise to zeros only
        for k in range(n_qubits):
            if images[-1][k] == 0:
                images[-1][k] = algorithm_globals.random.uniform(0, np.pi / 4)

    return images, labels

# -------------------------
# Build QCNN for arbitrary n_qubits (power of two), measuring the final pooled qubit
# -------------------------
def build_qcnn(estimator, n_qubits):
    _require_power_of_two(n_qubits)

    feature_map = ZFeatureMap(n_qubits)

    ansatz = QuantumCircuit(n_qubits, name="Ansatz")

    # "active" holds the absolute indices of the current working block
    active = list(range(n_qubits))
    stage = 1
    while len(active) > 1:
        # Convolution over the active block
        ansatz.compose(conv_layer(len(active), f"c{stage}"), active, inplace=True)

        # Pool first half into second half within the active block
        half = len(active) // 2
        sources_local = list(range(0, half))
        sinks_local   = list(range(half, len(active)))
        ansatz.compose(pool_layer(sources_local, sinks_local, f"p{stage}"), active, inplace=True)

        # Next stage works on the sinks (second half)
        active = active[half:]
        stage += 1

    readout_qubit = active[0]  # single remaining qubit
    # Construct observable with Z on readout, I elsewhere
    obs = ["I"] * n_qubits
    obs[readout_qubit] = "Z"
    observable = SparsePauliOp.from_list([("".join(obs), 1)])

    circuit = QuantumCircuit(n_qubits)
    circuit.compose(feature_map, range(n_qubits), inplace=True)
    circuit.compose(ansatz, range(n_qubits), inplace=True)

    qnn = EstimatorQNN(
        circuit=circuit.decompose(),
        observables=observable,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        estimator=estimator,
    )
    return qnn

# -------------------------
# One trial
# -------------------------
def run_trial(seed, n_qubits=8, num_images=200, maxiter=200, estimator=None, dataset=None,
              split_random_state=None, optimizer_kind="COBYLA"):
    _require_power_of_two(n_qubits)
    algorithm_globals.random_seed = int(seed)
    rng = np.random.default_rng(int(seed))

    # dataset (fresh or fixed)
    if dataset is None:
        images, labels = generate_dataset(num_images, n_qubits)
    else:
        images, labels = dataset

    rs = split_random_state if split_random_state is not None else int(seed)
    x_tr, x_te, y_tr, y_te = train_test_split(
        images, labels, test_size=0.30, random_state=rs, stratify=labels
    )

    qnn = build_qcnn(estimator or Estimator(), n_qubits)
    init = rng.uniform(-np.pi, np.pi, qnn.num_weights)

    # pick optimizer
    if optimizer_kind == "COBYLA":
        optimizer = COBYLA(maxiter=maxiter)
    else:
        try:
            from qiskit_algorithms.optimizers import SPSA, L_BFGS_B
        except Exception:
            from qiskit.algorithms.optimizers import SPSA, L_BFGS_B
        optimizer = {"SPSA": SPSA(maxiter=maxiter),
                     "L_BFGS_B": L_BFGS_B(maxiter=maxiter)}.get(optimizer_kind, COBYLA(maxiter=maxiter))

    clf = NeuralNetworkClassifier(qnn, optimizer=optimizer, initial_point=init, callback=None)

    xtr, ytr = np.asarray(x_tr), np.asarray(y_tr)
    clf.fit(xtr, ytr)

    train_acc = float(clf.score(xtr, ytr))
    xte, yte = np.asarray(x_te), np.asarray(y_te)
    test_acc  = float(clf.score(xte, yte))

    return {"seed": int(seed), "train_acc": train_acc, "test_acc": test_acc}

# -------------------------
# Many trials + histogram + CSV
# -------------------------
def run_many(n_trials=10, n_qubits=8, num_images=200, maxiter=200, vary_dataset=True, vary_split=True,
             optimizer_kind="COBYLA", save_csv_path=None, show_hist=True):
    _require_power_of_two(n_qubits)
    est = Estimator()
    results = []

    fixed_dataset = None
    if not vary_dataset:
        fixed_dataset = generate_dataset(num_images, n_qubits)

    t0 = time.time()
    for i in range(n_trials):
        seed = 10000 + i
        dataset = None if vary_dataset else fixed_dataset
        split_rs = (seed if vary_split else 246)
        res = run_trial(seed, n_qubits, num_images, maxiter, est, dataset, split_rs, optimizer_kind)
        results.append(res)
        print(f"[{i+1}/{n_trials}] seed={seed} train={res['train_acc']:.3f} test={res['test_acc']:.3f}")

    test_accs = np.array([r["test_acc"] for r in results])
    train_accs = np.array([r["train_acc"] for r in results])

    print(f"\nTrials: {n_trials} | elapsed: {time.time()-t0:.1f}s")
    print(f"Test acc  mean={test_accs.mean():.3f}  std={test_accs.std(ddof=1):.3f}  "
          f"min={test_accs.min():.3f}  max={test_accs.max():.3f}")
    print(f"Train acc mean={train_accs.mean():.3f}  std={train_accs.std(ddof=1):.3f}  "
          f"min={train_accs.min():.3f}  max={train_accs.max():.3f}")

    if show_hist:
        plt.figure(figsize=(6,4))
        plt.hist(test_accs, bins=10)
        plt.xlabel("Test Accuracy")
        plt.ylabel("Count")
        plt.title(f"Distribution of Test Accuracy (n={n_trials}, {n_qubits} qubits)")
        plt.show()

    # --- CSV save ---
    if save_csv_path:
        fieldnames = ["trial", "seed", "train_acc", "test_acc",
                      "n_qubits", "num_images", "maxiter", "vary_dataset", "vary_split", "optimizer"]
        with open(save_csv_path, "w", newline="") as f:
            w = csv.DictWriter(f, fieldnames=fieldnames)
            w.writeheader()
            for i, r in enumerate(results, start=1):
                w.writerow({
                    "trial": i,
                    "seed": r["seed"],
                    "train_acc": float(r["train_acc"]),
                    "test_acc": float(r["test_acc"]),
                    "n_qubits": int(n_qubits),
                    "num_images": int(num_images),
                    "maxiter": int(maxiter),
                    "vary_dataset": bool(vary_dataset),
                    "vary_split": bool(vary_split),
                    "optimizer": str(optimizer_kind),
                })
        print(f"Saved results to {save_csv_path}")

    return results


In [ ]:
results = run_many(
    n_trials=3, n_qubits=16, num_images=200, maxiter=50,
    vary_dataset=True, vary_split=True,
    save_csv_path="16qbit_50iter.csv"
)


No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.


[1/3] seed=10000 train=0.514 test=0.567


No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.


[2/3] seed=10001 train=0.529 test=0.600


No gradient function provided, creating a gradient function. If your Estimator requires transpilation, please provide a pass manager.
